##### Import libraries

In [208]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math

In [209]:
class data_preprocessing:
    def __init__(self,instance_path):
        self.instance_path=instance_path
        self.info, self.flights = self.read_file(f_name=self.instance_path)
        
        self.flights_by_day_dict = self.flights_by_day(flight_list=self.flights)
        self.airports_by_area = self.get_airports_by_areas()
        
        self.number_of_areas,self.starting_airport=int(self.info[0][0]),self.info[0][1]
        self.starting_area=self.associated_area_to_airport(airport=self.starting_airport)
        
        self.visited_areas=[self.associated_area_to_airport(self.starting_airport),"1","5"]
        
    def read_file(self,f_name):
        dist = []
        line_nu = -1
        with open(f_name) as infile:
            for line in infile:
                line_nu += 1
                if line_nu == 0:
                    index = int(line.split()[0]) * 2 + 1
                if line_nu >= index:
                    temp = line.split()
                    temp[2] = int(temp[2])
                    temp[3] = int(temp[3])
                    dist.append(temp)
                else:
                    dist.append(line.split())
            info = dist[0:int(dist[0][0])*2+1]
            flights = dist[int(dist[0][0])*2+1:]
        return info, flights
    
    def flights_by_day(self,flight_list):
        # Create an empty dictionary to hold flights organized by day
        flights_by_day = {}

        # Iterate over each flight in the input list
        for flight in flight_list:
            # Extract the day from the flight entry
            day = flight[2]

            # Create a flight entry without the day
            flight_without_day = flight[:2] + flight[3:]

            # Add the flight to the corresponding day in the dictionary
            if day not in flights_by_day:
                flights_by_day[day] = []
            flights_by_day[day].append(flight_without_day)
            
        return flights_by_day
    
    def get_cost(self, day, from_airport, to_airport):
        flights = self.flights_by_day_dict.get(day, [])
        for flight in flights:
            if flight[0] == from_airport and flight[1] == to_airport:
                return flight[2]
        return float('inf')
    
    def get_airports_by_areas(self):
        areas = {}
        area_num = int(self.info[0][0])
        for i in range(0, area_num):
            areas[f"{i}"] = self.info[2+i * 2]
        return areas
    
    def associated_area_to_airport(self,airport):
        for area, airports in self.airports_by_area.items():
            if airport in airports:
                return area
        return "Airport not found" 
    
    def possible_flights_from_an_airport_at_a_specific_day(self,day,from_airport):
        daily_flights = self.flights_by_day_dict.get(day, [])
        
        flights_from_airport = []
        for flight in daily_flights:
            if flight[0] == from_airport:
                
                destination_area = self.associated_area_to_airport(airport=flight[1])
                if destination_area not in self.visited_areas:
                    flights_from_airport.append(flight)

        return flights_from_airport

In [210]:
Data_Preprocessing=data_preprocessing(instance_path="Flight connections dataset/1.in")

In [211]:
class Node:
    def __init__(self, airport, area, day, parent=None, path_cost=0, visited_areas=None):
        self.airport = airport
        self.area = area
        self.day = day  # Track the current day
        self.parent = parent
        self.children = []
        self.visits = 0
        self.path_cost = path_cost
        self.value = 0
        self.visited_areas = visited_areas if visited_areas is not None else []

    def add_child(self, child):
        self.children.append(child)

    def update(self, value):
        self.visits += 1
        self.value += value

    def is_fully_expanded(self, total_areas):
        return len(self.visited_areas) >= total_areas

In [212]:
class MCTS:
    def __init__(self, data_processor):
        self.data_processor = data_processor
        self.root = Node(data_processor.starting_airport, data_processor.starting_area, 0, visited_areas=[data_processor.starting_area])

    def select(self, node):
        best_score = -float('inf')
        best_child = None
        for child in node.children:
            # Compute the UCB1 value for each child
            score = child.value / child.visits + math.sqrt(2 * math.log(node.visits) / child.visits)
            if score > best_score:
                best_score = score
                best_child = child
        return best_child

    def expand(self, node):
        # Expansion should consider the next day
        next_day = node.day + 1
        possible_flights = self.data_processor.possible_flights_from_an_airport_at_a_specific_day(next_day, node.airport)
        
        if possible_flights:
            chosen_flight = random.choice(possible_flights)
            new_area = self.data_processor.associated_area_to_airport(chosen_flight[1])
            if new_area not in node.visited_areas:
                new_node = Node(chosen_flight[1], new_area, next_day, node, node.path_cost + chosen_flight[2], node.visited_areas + [new_area])
                node.add_child(new_node)
                return new_node
        return None

    def simulate(self, node):
        cost = node.path_cost
        current_node = node
        # Simulate until all areas are visited or no more valid flights
        while not current_node.is_fully_expanded(self.data_processor.number_of_areas):
            next_day = current_node.day + 1
            flights = self.data_processor.possible_flights_from_an_airport_at_a_specific_day(next_day, current_node.airport)
            if not flights:
                break
            chosen_flight = random.choice(flights)
            new_area = self.data_processor.associated_area_to_airport(chosen_flight[1])
            if new_area not in current_node.visited_areas:
                cost += chosen_flight[2]
                current_node = Node(chosen_flight[1], new_area, next_day, current_node, cost, current_node.visited_areas + [new_area])
        return cost

    def backpropagate(self, node, value):
        while node:
            node.update(value)
            node = node.parent

    def search(self, iterations):
        self.root = Node(self.data_processor.starting_airport, self.data_processor.starting_area, 0, visited_areas=[self.data_processor.starting_area])
        for _ in range(iterations):
            leaf = self.select(self.root)
            if leaf is None:
                # No valid child to select, possibly all nodes are fully expanded
                # Could implement reselection or stop the search
                continue
            if not leaf.is_fully_expanded(self.data_processor.number_of_areas):
                new_leaf = self.expand(leaf)
                if new_leaf is None:
                    # Expansion was not possible; might be fully expanded or no valid flights
                    continue
                leaf = new_leaf
            if leaf:
                simulation_result = self.simulate(leaf)
                self.backpropagate(leaf, simulation_result)

In [213]:
mcts=MCTS(data_processor=Data_Preprocessing)

In [214]:
mcts.search(1000)  # Adjust iterations as needed

In [215]:
print(f'Optimal path cost found: {mcts.root.value}')

Optimal path cost found: 0
